In [1]:
import numpy as np
from tqdm import tqdm
import wandb as wb
import imageio
from io import BytesIO
import pickle
import zarr
from pathlib import Path
import os
import random

from src.codecs.imagecodecs import Jpeg2k, register_codecs

In [2]:
data_base_dir = Path(os.environ.get("FURNITURE_DATA_DIR", "data"))

## Inspect the current image data file we're using

In [3]:
datapath = data_base_dir / "processed/sim/image_small/one_leg/data.zarr"

store = zarr.open(datapath, mode="r")

In [4]:
store["color_image1"].shape, store["color_image1"].chunks

((370234, 224, 224, 3), (1, 224, 224, 3))

In [5]:
# Check the storage size of the dataset in GB
store["color_image1"].nbytes / 1e9

55.730583552